In [7]:
import re
import string

def clean_text(text):
    """
    Function to clean text data.
    :param text: The input string containing the text to be cleaned.
    :return: Cleaned text.
    """

    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

    return text

In [8]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/user/welzs0/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/user/welzs0/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/user/welzs0/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """
    Map POS tag to the first character lemmatize() accepts.
    """
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def tokenize_and_lemmatize(text):
    """
    Function to tokenize and lemmatize the input text.
    :param text: The input string containing the text to be processed.
    :return: A list of lemmatized tokens.
    """
    lemmatizer = WordNetLemmatizer()
    tokenized_text = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokenized_text]
    return lemmatized_text


In [10]:
def preprocess_text(text):
    cleaned_text = clean_text(text)
    preprocessed_text = tokenize_and_lemmatize(cleaned_text)
    
    return preprocessed_text

In [11]:
def clean_dataset(dataset):
    for element in tqdm(dataset, desc="Processing Text"):
        element['sentence'] = clean_text(element['sentence'])

In [12]:
def preprocess_dataset(dataset):
    for element in tqdm(dataset, desc="Processing Text"):
        element['sentence'] = preprocess_text(element['sentence'])

In [13]:
preprocess_dataset(rest_train)
preprocess_dataset(laptop_train)
preprocess_dataset(rest_test)
preprocess_dataset(laptop_test)

Processing Text: 100%|███████████████████████████████████████████████████████████████| 638/638 [00:00<00:00, 891.32it/s]


In [14]:
print(rest_train[0])

{'polarity': 'positive', 'term': 'server', 'id': '1592_0', 'sentence': ['our', 'server', 'be', 'very', 'helpful', 'and', 'friendly']}


### TFIDF with Logistic Regression

In [22]:
rest_train_sentences = [" ".join(item['sentence']) for item in rest_train]
rest_train_polarities = [item['polarity'] for item in rest_train]

rest_test_sentences = [" ".join(item['sentence']) for item in rest_test]
rest_test_polarities = [item['polarity'] for item in rest_test]

In [46]:
laptop_train_sentences = [" ".join(item['sentence']) for item in laptop_train]
laptop_train_polarities = [item['polarity'] for item in laptop_train]

laptop_test_sentences = [" ".join(item['sentence']) for item in laptop_test]
laptop_test_polarities = [item['polarity'] for item in laptop_test]

### Roberta Model

In [23]:
def load_dataset(split={"train", "test", "dev"}, domain={"rest", "laptop"}):
    with open(f'asc/{domain}/{split}.json', 'r') as file:
        dataset = json.load(file)
    return list(dataset.values())

In [24]:
rest_train = load_dataset(split="train", domain="rest") + load_dataset(split="dev", domain = "rest")
rest_test = load_dataset(split="test", domain="rest")
laptop_train = load_dataset(split="train", domain="laptop") + load_dataset(split="dev", domain = "laptop")
laptop_test = load_dataset(split="test", domain="laptop")

In [25]:
clean_dataset(rest_train)
clean_dataset(rest_test)
clean_dataset(laptop_train)
clean_dataset(laptop_test)

Processing Text: 100%|████████████████████████████████████████████████████████████| 638/638 [00:00<00:00, 127239.12it/s]


In [26]:
import torch

In [27]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [28]:
rest_train[0]

{'polarity': 'positive',
 'term': 'server',
 'id': '1592_0',
 'sentence': 'our server was very helpful and friendly'}

In [29]:
rest_train_sentences = [item['term'] + ' [SEP] ' + item['sentence'] for item in rest_train]

rest_test_sentences = [item['term'] + ' [SEP] ' + item['sentence'] for item in rest_test]
 
laptop_train_sentences = [item['term'] + ' [SEP] ' + item['sentence'] for item in laptop_train]

laptop_test_sentences = [item['term'] + ' [SEP] ' + item['sentence'] for item in laptop_test]

In [30]:
#rest_train_sentences = [" ".join(item['sentence']) for item in rest_train]
rest_train_polarities = [item['polarity'] for item in rest_train]

#rest_test_sentences = [" ".join(item['sentence']) for item in rest_test]
rest_test_polarities = [item['polarity'] for item in rest_test]


#laptop_train_sentences = [" ".join(item['sentence']) for item in laptop_train]
laptop_train_polarities = [item['polarity'] for item in laptop_train]

#laptop_test_sentences = [" ".join(item['sentence']) for item in laptop_test]
laptop_test_polarities = [item['polarity'] for item in laptop_test]

To get results with the different subsets (e.g. laptop of restaurant dataset) simply remove the unwanted subset from the cell below.

In [31]:
train_sentences = rest_train_sentences + laptop_train_sentences
train_polarities = rest_train_polarities + laptop_train_polarities

test_sentences = rest_test_sentences + laptop_test_sentences
test_polarities = rest_test_polarities + laptop_test_polarities

In [32]:
random_index = random.randint(0, len(train_sentences))
random_sentence = train_sentences[random_index]
random_polarity = train_polarities[random_index]
print(random_sentence)
print(random_polarity)

DC jack [SEP] then ive fixed the dc jack inside the unit rewired the dc jack to the outside of the laptop replaced the power brick
neutral


In [33]:
pos = 0
neg = 0
neu = 0
for polarity in train_polarities:
    if polarity == 'positive':
        pos += 1
    elif polarity == 'negative':
        neg += 1
    elif polarity == 'neutral':
        neu += 1
print(f"Positive : {pos}")
print(f"Negative : {neg}")
print(f"Neutral : {neu}")

Positive : 3151
Negative : 1671
Neutral : 1093


In [51]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_polarities)
y_test = label_encoder.transform(test_polarities)

['negative' 'neutral' 'positive']


In [35]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

In [36]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

#### Kinds of Roberta Model we can try:
- RoBERTa Base Model
- RoBERTa Large Model
- distilroberta-base
- Visit Hugging Face for more....

In [37]:
from transformers import BertTokenizer
model_name = 'roberta-base'
# Load the BERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)

def encode_sentences(sentences, max_length=512):
    input_ids = []
    attention_masks = []

    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
                            sentence,                      # Sentence to encode
                            add_special_tokens=True,       # Add '[CLS]' and '[SEP]'
                            max_length=max_length,         # Pad & truncate all sentences
                            padding='max_length',          # Pad all sentences to max length
                            truncation=True,               # Explicitly truncate to max length
                            return_attention_mask=True,    # Construct attention masks
                            return_tensors='pt',           # Return pytorch tensors
                        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Encode the sentences (X_train and X_test)
train_inputs, train_masks = encode_sentences(train_sentences)
test_inputs, test_masks = encode_sentences(test_sentences)

In [38]:
train_labels = torch.tensor(y_train)
test_labels = torch.tensor(y_test)

In [39]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16  # Adjust this according to your GPU capacity

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [40]:
model = RobertaForSequenceClassification.from_pretrained(
    model_name,  # Use the 12-layer BERT model, with an uncased vocab
    num_labels=3,        # Number of output labels (3 for positive/negative/neutral)
    output_attentions=False,
    output_hidden_states=False,
)

# Tell the model to run on GPU
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [44]:
from transformers import get_linear_schedule_with_warmup
import torch
# Implement the training loop
epochs = 5
# Define the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Total number of training steps
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [45]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import numpy as np

In [46]:
best_accuracy = 0
for epoch in range(0, epochs):
    # Training step
    model.train()
    total_train_loss = 0
    progress_bar = tqdm(train_dataloader, desc="Epoch {:1d}".format(epoch+1), leave=False, disable=False)
    for batch in progress_bar:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear previously calculated gradients
        model.zero_grad()        

        # Perform a forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        # Perform a backward pass
        loss.backward()

        # Update parameters and take a step using the computed gradient
        optimizer.step()
        scheduler.step()
        # Update the progress bar
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
    # Calculate the average loss over the training data.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    print(f"  Average training loss: {avg_train_loss:.2f}")
    
    model.eval()

    # Initialize variables to gather predictions and true labels
    all_predictions = []
    all_true_labels = []

    with torch.no_grad():
        total_eval_loss = 0
        for batch in tqdm(test_dataloader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

            loss = outputs.loss
            total_eval_loss += loss.item()

            logits = outputs.logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Store predictions and true labels
            all_predictions.extend(np.argmax(logits, axis=1).flatten())
            all_true_labels.extend(label_ids.flatten())
    precision, recall, f1_score, _ = precision_recall_fscore_support(all_true_labels, all_predictions, average='weighted')
    accuracy = accuracy_score(all_true_labels, all_predictions)
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-Score: {f1_score:.4f}')
    if accuracy >= best_accuracy:
        print(f"  Accuracy increased from {accuracy:.2f} to {best_accuracy:.2f}, saving model.")
        best_accuracy = accuracy
        best_model_state = model.state_dict()
torch.save(best_model_state, 'best_BERT_model.bin')

  Average training loss: 0.26


100%|█████████████████████████████████████████████████████████████████████████████████| 110/110 [00:12<00:00,  8.51it/s]


Accuracy: 0.8498
Precision: 0.8522
Recall: 0.8498
F1-Score: 0.8491
  Accuracy increased from 0.85 to 0.00, saving model.


  Average training loss: 0.17


100%|█████████████████████████████████████████████████████████████████████████████████| 110/110 [00:12<00:00,  8.50it/s]


Accuracy: 0.8510
Precision: 0.8482
Recall: 0.8510
F1-Score: 0.8430
  Accuracy increased from 0.85 to 0.85, saving model.


  Average training loss: 0.10


100%|█████████████████████████████████████████████████████████████████████████████████| 110/110 [00:12<00:00,  8.48it/s]


Accuracy: 0.8589
Precision: 0.8553
Recall: 0.8589
F1-Score: 0.8554
  Accuracy increased from 0.86 to 0.85, saving model.


KeyboardInterrupt: 

In [57]:
from sklearn.metrics import precision_score
model.eval()

# Initialize variables to gather predictions and true labels
all_predictions = []
all_true_labels = []

with torch.no_grad():
    total_eval_loss = 0
    for batch in tqdm(test_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    
        loss = outputs.loss
        total_eval_loss += loss.item()
        
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Store predictions and true labels
        all_predictions.extend(np.argmax(logits, axis=1).flatten())
        all_true_labels.extend(label_ids.flatten())
precision, recall, f1_score, _ = precision_recall_fscore_support(all_true_labels, all_predictions, average='weighted')
#0 neg, 1 neut, 2 pos
accuracy = accuracy_score(all_true_labels, all_predictions)
conf_matrix = confusion_matrix(all_true_labels, all_predictions)
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1_score:.4f}')
print('Confusion Matrix:\n', conf_matrix)


100%|█████████████████████████████████████████████████████████████████████████████████| 110/110 [00:12<00:00,  8.79it/s]

Accuracy: 0.8544
Precision: 0.8497
Recall: 0.8544
F1-Score: 0.8499
Confusion Matrix:
 [[ 265   35   24]
 [  55  223   87]
 [  20   35 1014]]


NameError: name 'cm' is not defined

In [61]:
TP = conf_matrix[2, 2]
FP = np.sum(conf_matrix[:, 2]) - TP
precision_positve = TP / (TP + FP)


TP = conf_matrix[0, 0]
FP = np.sum(conf_matrix[:, 0]) - TP
precision_negative = TP / (TP + FP)

TP = conf_matrix[1, 1]
FP = np.sum(conf_matrix[:, 1]) - TP
precision_neutral = TP / (TP + FP)
print(f"Precision Positive {precision_positve}")
print(f"Precision Negative {precision_negative}")
print(f"Precision Neutral {precision_neutral}")

Precision Positive 0.9013333333333333
Precision Negative 0.7794117647058824
Precision Neutral 0.7610921501706485
